# 10X `tcellmatch` Tutorial - preprocessing

This tutorial shows how to preprocess the raw data files from 10x database and feed into a feed forward network for categorical classification. The introduction of 10x can be found here: https://www.10xgenomics.com

The task of this model is based on [this paper](https://www.embopress.org/doi/full/10.15252/msb.20199416), but, in brief, it approximates CDR3 RNA sequences to antigen specificity. This antigen specificity can further be predicted as either number of bindings or max bindings for each sequence.

See the ``README.md`` file for details on different architectures.


In [5]:
import pandas as pd
import tcellmatch.api as tm
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from pytorch_model_summary import summary
from torchmetrics import Accuracy
import torch
import os
import numpy as np

# Get Data Directories

You can find the data below:

https://www.10xgenomics.com/resources/datasets/cd-8-plus-t-cells-of-healthy-donor-1-1-standard-3-0-2

https://www.10xgenomics.com/resources/datasets/cd-8-plus-t-cells-of-healthy-donor-2-1-standard-3-0-2

For each donor, the binarized matrix corresponds to "binarized matrix CSV" on 10X and the clonotype matrix corresponds to "VDJ - Clonotype info (CSV)"

In [2]:
# Path of input directory.
indir = '../tutorial_data/'
# Path to 10x raw files.
fns = [f"{indir}vdj_v1_hs_aggregated_donor1_binarized_matrix.csv",
       f"{indir}vdj_v1_hs_aggregated_donor2_binarized_matrix.csv"]
# Path to preprocessed clonotypes files.
fns_clonotype = [f"{indir}vdj_v1_hs_aggregated_donor1_clonotypes.csv",
                 f"{indir}vdj_v1_hs_aggregated_donor2_clonotypes.csv"]

In [3]:
cellranger_out = pd.read_csv(fns[0])
cellranger_out.head()

FileNotFoundError: [Errno 2] No such file or directory: '../tutorial_data/vdj_v1_hs_aggregated_donor1_binarized_matrix.csv'

In [ ]:
data = cellranger_out
column_names = data.columns
column_types = data.dtypes

In [ ]:
cellranger_out = pd.read_csv(fns_clonotype[0])
cellranger_out.head()

## List of column names of labels to predict in 10x raw files
Here we take all antigens from 10x dataset for prediction

In [ ]:
target_ids = [
    'A0101_VTEHDTLLY_IE-1_CMV_binder',
    'A0201_KTWGQYWQV_gp100_Cancer_binder',
    'A0201_ELAGIGILTV_MART-1_Cancer_binder',
    'A0201_CLLWSFQTSA_Tyrosinase_Cancer_binder',
    'A0201_IMDQVPFSV_gp100_Cancer_binder',
    'A0201_SLLMWITQV_NY-ESO-1_Cancer_binder',
    'A0201_KVAELVHFL_MAGE-A3_Cancer_binder',
    'A0201_KVLEYVIKV_MAGE-A1_Cancer_binder',
    'A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase_binder',
    'A0201_LLDFVRFMGV_EBNA-3B_EBV_binder',
    'A0201_LLMGTLGIVC_HPV-16E7_82-91_binder',
    'A0201_CLGGLLTMV_LMP-2A_EBV_binder',
    'A0201_YLLEMLWRL_LMP1_EBV_binder',
    'A0201_FLYALALLL_LMP2A_EBV_binder',
    'A0201_GILGFVFTL_Flu-MP_Influenza_binder',
    'A0201_GLCTLVAML_BMLF1_EBV_binder',
    'A0201_NLVPMVATV_pp65_CMV_binder',
    'A0201_ILKEPVHGV_RT_HIV_binder',
    'A0201_FLASKIGRLV_Ca2-indepen-Plip-A2_binder',
    'A2402_CYTWNQMNL_WT1-(235-243)236M_Y_binder',
    'A0201_RTLNAWVKV_Gag-protein_HIV_binder',
    'A0201_KLQCVDLHV_PSA146-154_binder',
    'A0201_LLFGYPVYV_HTLV-1_binder',
    'A0201_SLFNTVATL_Gag-protein_HIV_binder',
    'A0201_SLYNTVATLY_Gag-protein_HIV_binder',
    'A0201_SLFNTVATLY_Gag-protein_HIV_binder',
    'A0201_RMFPNAPYL_WT-1_binder',
    'A0201_YLNDHLEPWI_BCL-X_Cancer_binder',
    'A0201_MLDLQPETT_16E7_HPV_binder',
    'A0301_KLGGALQAK_IE-1_CMV_binder',
    'A0301_RLRAEAQVK_EMNA-3A_EBV_binder',
    'A0301_RIAAWMATY_BCL-2L1_Cancer_binder',
    'A1101_IVTDFSVIK_EBNA-3B_EBV_binder',
    'A1101_AVFDRKSDAK_EBNA-3B_EBV_binder',
    'B3501_IPSINVHHY_pp65_CMV_binder',
    'A2402_AYAQKIFKI_IE-1_CMV_binder',
    'A2402_QYDPVAALF_pp65_CMV_binder',
    'B0702_QPRAPIRPI_EBNA-6_EBV_binder',
    'B0702_TPRVTGGGAM_pp65_CMV_binder',
    'B0702_RPPIFIRRL_EBNA-3A_EBV_binder',
    'B0702_RPHERNGFTVL_pp65_CMV_binder',
    'B0801_RAKFKQLL_BZLF1_EBV_binder',
    'B0801_ELRRKMMYM_IE-1_CMV_binder',
    'B0801_FLRGRAYGL_EBNA-3A_EBV_binder',
    'A0101_SLEGGGLGY_NC_binder',
    'A0101_STEGGGLAY_NC_binder',
    'A0201_ALIAPVHAV_NC_binder',
    'A2402_AYSSAGASI_NC_binder',
    'B0702_GPAESAAGL_NC_binder',
    'NR(B0801)_AAKGRGAAL_NC_binder'
]

## List of column names of negative controls in 10x raw files

In [ ]:
nc_cols = [
    'A0101_SLEGGGLGY_NC_binder',
    'A0101_STEGGGLAY_NC_binder',
    'A0201_ALIAPVHAV_NC_binder',
    'A2402_AYSSAGASI_NC_binder',
    'B0702_GPAESAAGL_NC_binder',
    'NR(B0801)_AAKGRGAAL_NC_binder'
]

### (Optional) Use these if you want binding counts instead of top binder categorical

In [ ]:
USE_BIND_COUNTS = True

if USE_BIND_COUNTS:
    target_ids = [
        'A0101_VTEHDTLLY_IE-1_CMV',
        'A0201_KTWGQYWQV_gp100_Cancer',
        'A0201_ELAGIGILTV_MART-1_Cancer',
        'A0201_CLLWSFQTSA_Tyrosinase_Cancer',
        'A0201_IMDQVPFSV_gp100_Cancer',
        'A0201_SLLMWITQV_NY-ESO-1_Cancer',
        'A0201_KVAELVHFL_MAGE-A3_Cancer',
        'A0201_KVLEYVIKV_MAGE-A1_Cancer',
        'A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase',
        'A0201_LLDFVRFMGV_EBNA-3B_EBV',
        'A0201_LLMGTLGIVC_HPV-16E7_82-91',
        'A0201_CLGGLLTMV_LMP-2A_EBV',
        'A0201_YLLEMLWRL_LMP1_EBV',
        'A0201_FLYALALLL_LMP2A_EBV',
        'A0201_GILGFVFTL_Flu-MP_Influenza',
        'A0201_GLCTLVAML_BMLF1_EBV',
        'A0201_NLVPMVATV_pp65_CMV',
        'A0201_ILKEPVHGV_RT_HIV',
        'A0201_FLASKIGRLV_Ca2-indepen-Plip-A2',
        'A2402_CYTWNQMNL_WT1-(235-243)236M_Y',
        'A0201_RTLNAWVKV_Gag-protein_HIV',
        'A0201_KLQCVDLHV_PSA146-154',
        'A0201_LLFGYPVYV_HTLV-1',
        'A0201_SLFNTVATL_Gag-protein_HIV',
        'A0201_SLYNTVATLY_Gag-protein_HIV',
        'A0201_SLFNTVATLY_Gag-protein_HIV',
        'A0201_RMFPNAPYL_WT-1',
        'A0201_YLNDHLEPWI_BCL-X_Cancer',
        'A0201_MLDLQPETT_16E7_HPV',
        'A0301_KLGGALQAK_IE-1_CMV',
        'A0301_RLRAEAQVK_EMNA-3A_EBV',
        'A0301_RIAAWMATY_BCL-2L1_Cancer',
        'A1101_IVTDFSVIK_EBNA-3B_EBV',
        'A1101_AVFDRKSDAK_EBNA-3B_EBV',
        'B3501_IPSINVHHY_pp65_CMV',
        'A2402_AYAQKIFKI_IE-1_CMV',
        'A2402_QYDPVAALF_pp65_CMV',
        'B0702_QPRAPIRPI_EBNA-6_EBV',
        'B0702_TPRVTGGGAM_pp65_CMV',
        'B0702_RPPIFIRRL_EBNA-3A_EBV',
        'B0702_RPHERNGFTVL_pp65_CMV',
        'B0801_RAKFKQLL_BZLF1_EBV',
        'B0801_ELRRKMMYM_IE-1_CMV',
        'B0801_FLRGRAYGL_EBNA-3A_EBV',
        'A0101_SLEGGGLGY_NC',
        'A0101_STEGGGLAY_NC',
        'A0201_ALIAPVHAV_NC',
        'A2402_AYSSAGASI_NC',
        'B0702_GPAESAAGL_NC',
        'NR(B0801)_AAKGRGAAL_NC',
    ]
    nc_cols = [
        'A0101_SLEGGGLGY_NC',
        'A0101_STEGGGLAY_NC',
        'A0201_ALIAPVHAV_NC',
        'A2402_AYSSAGASI_NC',
        'B0702_GPAESAAGL_NC',
        'NR(B0801)_AAKGRGAAL_NC'
    ]

# Read data

## Create model object
EstimatorFfn() includes all of reading, training and testing modules.

In [ ]:
ffn = tm.models.EstimatorFfn()

## Read 10x raw files, taking out TCR CDR3 and binarized binding matrix as training data
We encode the TCR CDR3 amino acid sequences (include TRA and TRB) with one-hot encoding, the embedded sequences are of shape [num_samples, tra/trb, max_sequence_length, aa_onehot_dim]. For example if we take out 4000 TRB sequences seperately, the maximal length of sequences is 30 and we have 22 amino acids, the shape of output would be [4000, 1, 30, 26]. 

In [ ]:
ffn.read_binarized_matrix(
    fns=[indir + x for x in fns],
    fns_clonotype=[indir + x for x in fns_clonotype],
    fns_covar=[],
    fn_blosum=f"{indir}BLOSUM50.csv",
    blosum_encoding=False,
    is_train=True,
    # include categorical variable for which donor we get from
    covariate_formula_categ=["donor"],
    covariate_formula_numeric=[],
    # Whether to add an additional non-binder category for softmax activation function
    add_non_binder_for_softmax=False,
    # we are only keeping trb chain
    chains="trb",
    label_cols=target_ids,
    nc_cols=nc_cols
)
sums_across_last_dim = np.sum(ffn.x_train, axis=-1)

# Find rows which are not "zero-hot"
non_zero_hot_rows = np.any(sums_across_last_dim > 0, axis=-1)

ffn.x_train = ffn.x_train[non_zero_hot_rows]
# valid idxs along the first dimension
valid_idxs = non_zero_hot_rows[:, 0, 0]
ffn.clone_train = ffn.clone_train[valid_idxs]
ffn.covariates_train = ffn.covariates_train[valid_idxs]
ffn.y_train = ffn.y_train[valid_idxs]

ffn.x_train = ffn.x_train[:, np.newaxis, :]

The input consists of TCR CDR3 sequences and covariates. Covariates act as a additional information which will be concatenated with TCR CDR3 sequences during training. The target set is a binarized matrix which shows the binding between TCR CDR3 and antigens.

In [ ]:
print("Shape of TCR sequences: ",ffn.x_train.shape)
# print("The head of TCR sequences: ",ffn.x_train[0])
print("Shape of covariates: ",ffn.covariates_train.shape)
# print("The head of covariates: ",ffn.covariates_train[0:5])
print("Shape of target set: ",ffn.y_train.shape)
# print("The head of target set: ",ffn.y_train[0:5])

Some of these sequences are not one-hot, but "zero-hot" encoded. We remove those

## Downsample clonotypes to data stored in x_train
This avoids training, evaluation or test set being too biased toward a subset of TCRs.

In [ ]:
# max_obs: Maximum number of observations per clonotype.
ffn.downsample_clonotype(max_obs=10)

In [ ]:
print("Shape of TCR sequences: ",ffn.x_train.shape)
print("Shape of covariates: ",ffn.covariates_train.shape)
print("Shape of target set: ",ffn.y_train.shape)

## Create test dataset
We can either split the training set or use a new database as the test set. Here, we split test set from training set.

In [ ]:
ffn.clear_test_data()
ffn.sample_test_set(test_split=0.1)

In [ ]:
print("Shape of TCR CDR3 sequences for training: ",ffn.x_train.shape)
print("Shape of covariates for training: ",ffn.covariates_train.shape)
print("Shape of target set for training: ",ffn.y_train.shape)
print("Shape of TCR CDR3 sequences for test: ",ffn.x_test.shape)
print("Shape of covariates for test: ",ffn.covariates_test.shape)
print("Shape of target set for test: ",ffn.y_test.shape)

## Padding TCR CDR3 sequences in both training and testing set
Since we can use TCR CDR3 in another database as the test set, we should make sure they have same size.

In [ ]:
ffn.pad_sequence(target_len=40, sequence="tcr")

## Downsample data to given number of observations.
In order to save time, we sample a small dataset for training. Never use this method in practice. 

In [ ]:
# ffn.downsample_data(n=200, data="train")
# ffn.downsample_data(n=200, data="test")

In [ ]:
print("Shape of TCR CDR3 sequences for training: ",ffn.x_train.shape)
print("Shape of TCR CDR3 sequences for test: ",ffn.x_test.shape)

Data contains some rows with all zeros. We remove those rows. Additionally, we reshape data to comply with the models' requirements.

In [ ]:
sums_across_last_dim = np.sum(ffn.x_train, axis=-1) 
# Find rows which are not "zero-hot"
non_zero_hot_rows = np.any(sums_across_last_dim > 0, axis=-1).reshape(-1,)

In [ ]:
ffn.x_train = ffn.x_train[non_zero_hot_rows]
ffn.y_train = ffn.y_train[non_zero_hot_rows]
ffn.covariates_train = ffn.covariates_train[non_zero_hot_rows]

### Save imported data as NumPy Arrays
Once you have saved the data, you can reuse it in ``10x_dataset_training.ipynb`` without having to read in the data over and over. 

In [ ]:
SAVE_TO_NUMPY = True
if SAVE_TO_NUMPY:
    np.savez_compressed(
        f"{indir}ffn_data_continuous.npz",
        x_train=ffn.x_train,
        covariates_train=ffn.covariates_train,
        y_train=ffn.y_train,
        x_test=ffn.x_test,
        covariates_test=ffn.covariates_test,
        y_test=ffn.y_test,
        clone_train=ffn.clone_train
    )
    ffn.save_idx(f"{indir}SAVED_IDX")